<a href="https://colab.research.google.com/github/mmovahed/01_Knapsack/blob/main/MILP_DRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Mixed Integer Linear Programming (MILP) models using Deep Reinforcement Learning (DRL)

Let's describe and formulate the simple mixed integer linear programming (MILP) problem for steel blending that we discussed earlier.
Problem Description:
You want to blend steels with various chemical compositions to obtain 25 tons of steel with a specific chemical composition. The result should have 5% carbon and 5% molybdenum by weight, meaning 25 tons * 5% = 1.25 tons of carbon and 1.25 tons of molybdenum. The objective is to minimize the cost for blending the steel.
Variables:
- ( $x_i$ ): Binary variables indicating whether you purchase ingot ( $i$ ) (1 if yes, 0 if no).
- ( $y_j$ ): Continuous variables representing the quantity in tons of alloy ( $j$ ) that you purchase.
- ( $z$ ): Continuous variable representing the quantity of scrap steel that you purchase.
Parameters:
- ( $w_i$ ): Weight in tons of ingot ( $i$ ).
- ( $c_i$ ): Percentage of carbon in ingot ( $i$ ).
- ( $m_i$ ): Percentage of molybdenum in ingot ( $i$ ).
- ( $p_i$ ): Cost per ton of ingot ( $i$ ).
- ( $a_j$ ): Percentage of carbon in alloy ( $j$ ).
- ( $b_j$ ): Percentage of molybdenum in alloy ( $j$ ).
- ( $q_j$ ): Cost per ton of alloy ( $j$ ).
- ( $s_c$ ): Percentage of carbon in scrap steel.
- ( $s_m$ ): Percentage of molybdenum in scrap steel.
- ( $s_p$ ): Cost per ton of scrap steel.
Objective Function:
Minimize the total cost of purchasing ingots, alloys, and scrap steel.
$$ \text{Minimize} \quad Z = \sum_{i=1}^{4} p_i x_i + \sum_{j=1}^{3} q_j y_j + s_p z $$

Constraints:
- The total weight of the blend must be 25 tons.
$$ \sum_{i=1}^{4} w_i x_i + \sum_{j=1}^{3} y_j + z = 25 $$
- The total weight of carbon in the blend must be 1.25 tons.
$$ \sum_{i=1}^{4} \frac{c_i}{100} w_i x_i + \sum_{j=1}^{3} \frac{a_j}{100} y_j + \frac{s_c}{100} z = 1.25 $$
- The total weight of molybdenum in the blend must be 1.25 tons.
$$ \sum_{i=1}^{4} \frac{m_i}{100} w_i x_i + \sum_{j=1}^{3} \frac{b_j}{100} y_j + \frac{s_m}{100} z = 1.25 $$

Bounds:
- ( $x_i$ ) are binary variables, so ( $x_i \in {0, 1}$ ).
- ( $y_j$ ) and ( $z$ ) are non-negative.
This formulation captures the essence of the steel blending problem where you have to decide on the quantities of different types of steel to purchase in order to meet the desired chemical composition at the minimum cost. The binary variables ( $x_i$ ) ensure that you either purchase an entire ingot or not at all, while the continuous variables ( $y_j$ ) and ( $z$ ) allow for fractional amounts of alloys and scrap steel to be used in the blend.

In [144]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np

# Define the environment for the MILP problem
class MILPEnv:
    def __init__(self):
        # Define the problem parameters
        self.ingots = np.array([[5, 5, 3, 350],
                                [3, 4, 3, 330],
                                [4, 5, 4, 310],
                                [6, 3, 4, 280]])
        self.alloys = np.array([[8, 6, 500],
                                [7, 7, 450],
                                [6, 8, 400]])
        self.scrap = np.array([3, 9, 100])
        self.target_weight = 25
        self.target_composition = np.array([5, 5])  # 5% carbon, 5% molybdenum
        self.state = None

    def reset(self):
        # Reset the environment to the initial state
        self.state = np.zeros(8)  # Initial quantities of ingots and alloys
        return self.state

    def step(self, action):
        # Apply the action to the environment
        # For simplicity, let's say action is the index to increase the quantity by 1
        if action < 4:
            self.state[action] += 1  # Buy one more ingot
        else:
            self.state[action] += 0.1  # Buy 0.1 tons more of alloy or scrap

        # Calculate the current weight and composition
        weight = np.sum(self.ingots[:, 0] * self.state[:4]) + np.sum(self.alloys[:, 0] * self.state[4:7]) + self.scrap[0] * self.state[7]
        composition = np.array([
            np.sum(self.ingots[:, 1] * self.state[:4]) + np.sum(self.alloys[:, 1] * self.state[4:7]) + self.scrap[1] * self.state[7],
            np.sum(self.ingots[:, 2] * self.state[:4]) + np.sum(self.alloys[:, 2] * self.state[4:7]) + self.scrap[2] * self.state[7]
        ])

        # Check if the target weight and composition are met
        done = weight >= self.target_weight and np.all(composition >= self.target_composition * self.target_weight / 100)
        reward = -1 if not done else 1000 - np.sum(self.ingots[:, 3] * self.state[:4]) - np.sum(self.alloys[:, 2] * self.state[4:7]) - self.scrap[2] * self.state[7]

        return self.state, reward, done

# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        # Define the layers of the network
        self.fc = nn.Linear(8, 128)  # 8 inputs for the quantities of ingots and alloys
        self.action_head = nn.Linear(128, 8)  # 8 actions corresponding to the quantities

    def forward(self, x):
        # Forward pass through the network
        x = torch.relu(self.fc(x))
        action_probs = torch.softmax(self.action_head(x), dim=-1)
        return action_probs

# Initialize the environment and the policy network
env = MILPEnv()
policy_net = PolicyNetwork()

# Define the optimizer
optimizer = optim.Adam(policy_net.parameters(), lr=0.01)

# Training loop
num_episodes = 1000
max_steps_per_episode = 100
total_cost = 0
optimal_state = env.reset()

for episode in range(num_episodes):
    state = env.reset()
    for t in range(max_steps_per_episode):
        # Convert state to PyTorch tensor
        state_tensor = torch.from_numpy(state).float().unsqueeze(0)

        # Get action probabilities from the policy network
        action_probs = policy_net(state_tensor)
        m = Categorical(action_probs)

        # Sample an action from the distribution
        action = m.sample()

        # Take the action in the environment
        next_state, reward, done = env.step(action.item())

        # Calculate the loss
        loss = -m.log_prob(action) * reward
        total_cost += -reward  # Assuming reward is negative cost
        # Update the policy network
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if done:
            break

        state = next_state
        optimal_state = next_state
    if episode % 100 == 0:
        print(f'Reward in episode {episode} is {reward}')
        print(f'Loss in episode {episode} is {loss[0]}')

print(f"Optimal solution found: {optimal_state}")
print(f"Total cost: {total_cost}")

Reward in episode 0 is -990.0
Loss in episode 0 is -2792.54345703125
Reward in episode 100 is -680.0000000000005
Loss in episode 100 is -8.106232417048886e-05
Reward in episode 200 is -680.0000000000005
Loss in episode 200 is -8.106232417048886e-05
Reward in episode 300 is -680.0000000000005
Loss in episode 300 is -8.106232417048886e-05
Reward in episode 400 is -680.0000000000005
Loss in episode 400 is -8.106232417048886e-05
Reward in episode 500 is -680.0000000000005
Loss in episode 500 is -8.106232417048886e-05
Reward in episode 600 is -680.0000000000005
Loss in episode 600 is -8.106232417048886e-05
Reward in episode 700 is -680.0000000000005
Loss in episode 700 is -8.106232417048886e-05
Reward in episode 800 is -680.0000000000005
Loss in episode 800 is -8.106232417048886e-05
Reward in episode 900 is -680.0000000000005
Loss in episode 900 is -8.106232417048886e-05
Optimal solution found: [0.  0.  0.  0.  0.  0.  4.2 0. ]
Total cost: 720378.0
